# Baking Extended Events Recipes Without Breaking a Sweat

For each recipe you need:

- A session to capture the events
- A target configuration 
- A client application to read the data

The main ingredient is always [XESmartTarget](https://github.com/spaghettidba/XESmartTarget/), an open source tool to automate processing of Extened Events read from the streaming API.


In [ ]:
clear
$Server = "YourServerNameGoesHere"
$Path = "FolderThatContainsThisFile"
Invoke-SqlCmd -ServerInstance $Server -InputFile $Path\00_Setup.sql

In [ ]:
## Run a test workload on this computer
start-process "c:\ds3\driver_demo.cmd"

## Recipe1: Write to a CSV file and monitor in realtime with VSCode

In this demo I try to do the same I would do in SSMS with the "watch live data" window, displaying the events received from the server.

I would like to capture the execution events that I get in Profiler when I start the Standard template.

**Session**: Profiler Standard

I don't need to create the session, because dbatools can do that for me. There are a lot of handy functions in dbatools to work with Extended Events, including a function to create a session from a template. 

Chrissy LeMaire gathered a lot of useful session definitions from community blog posts and scripts, and templates from Microsoft tools, then she included those definitions in dbatools.

In order to see a list of available templates, all you have to do is run this command:


In [ ]:
Get-DbaXESessionTemplate | Out-GridView

In this case we can use the "Profiler Standard" template:

In [ ]:
clear
# Create and start the session using dbatools
Import-DbaXESessionTemplate -SqlInstance $Server -Name "Recipe01" -Template "Profiler Standard"
Start-DbaXESession -Session "Recipe01" -SqlInstance $Server

Now I have a session running and I could work with the events it captures. If I wanted to write that data to a file, I could use the file target and let it handle everything with the native tools.

One of the potential problems with this approach is that the data gets written to the disks of the server, which is not always feasible or recommended. 

With XESmartTarget I can write the data to a file on the client, using CSV as the output format.

What I need is a configuration for XESmartTarget that uses a [CsvAppenderResponse](https://github.com/spaghettidba/XESmartTarget/wiki/CsvAppenderResponse) to write the data according to my needs.

**Target:** [CsvAppenderResponse](https://github.com/spaghettidba/XESmartTarget/wiki/CsvAppenderResponse)

In [ ]:
# Display the json configuration file
code $Path\Recipe_01_Output_CSV.json

In [ ]:
# Start XESmartTarget
Start-Process $env:ProgramFiles\XESmartTarget\xesmarttarget.exe -ArgumentList @("--File", "$Path\Recipe_01_Output_CSV.json", "--GlobalVariables","ServerName=$Server")

In [ ]:
# Show the file in VSCode
code c:\temp\output.csv

## Recipe2: Write activity to a database table

One of the things you cannot achieve easily with the standard targets in Extended Events is writing to a database table. 

If you check the documentation and all the blogs that you find on the Internet, all of them recommend using two possible methods to extract the information from the session:

1. Shredding the XML of the session data from the ring buffer target
2. Using `master.sys.fn_xe_file_target_read_file` to read the session data from a file target

The first approach is extremely painful. XML shredding - enough said.

The second approach requires a file target, which is not always available. It also requires to stop the session to read all the data, or write extremely complicated code to read the data incrementally while the session is running (believe me, I did it and my brain still hurts).

This is where XESmartTarget jumps in to help you! All you have to do is write some configuration in the json file and let XESmartTarget do its magic.

For this task we can use a [TableAppenderResponse](https://github.com/spaghettidba/XESmartTarget/wiki/TableAppenderResponse), that takes care of reading all the events from the session using the streaming API and write the to a target table, that can also be created automatically from XESmartTarget itself.

**Session:**: Recipe02. Captures rpc_completed and sql_batch_completed events



In [ ]:
# Display the .sql script for the the session
code $Path\Recipe_02_Session_Setup.sql

In [ ]:
# Start the session
Invoke-DbaQuery -SqlInstance $Server -File "$Path\Recipe_02_Session_Setup.sql"

**Target**:  [TableAppenderResponse](https://github.com/spaghettidba/XESmartTarget/wiki/TableAppenderResponse)

Writes the data to the specified Server, Database, Schema and table.

In [ ]:
# Display the json configuration file
code $Path\Recipe_02_Output_Table.json

In [ ]:
# Start XESmartTarget
Start-Process $env:ProgramFiles\XESmartTarget\xesmarttarget.exe -ArgumentList @("--File", "$Path\Recipe_02_Output_Table.json", "--GlobalVariables","ServerName=$Server")

In [ ]:
# Display the data being captured
Invoke-Item "$Path\Recipe_02_Display_Data.sql"

## Recipe3: Merge and manipulate events before writing to a database table

In the previous recipe, each event used field and action names to map to the column names in the table. The same information (the text of the command) was stored in two separate columns: 

- `batch_text` for `sql_batch_completed` events
- `statement` for `rpc_completed` events

It would be nice if I could merge the two fields in a single column. XESmartTarget has a solution for that too.

**Session**:
I can reuse the previous session for this recipe. To avoid the confusion I can make a copy of the session and name it Recipe04. Again, dbatools has got me covered.

In [ ]:
clear
Get-DbaXESession -SqlInstance $Server -Session Recipe02 |
    Export-DbaXESessionTemplate -Path C:\temp\xe |
    Import-DbaXESessionTemplate  -SqlInstance $Server -Name "Recipe03"
Start-DbaXESession -SqlInstance $Server -Session Recipe03

**Target**:  [TableAppenderResponse](https://github.com/spaghettidba/XESmartTarget/wiki/TableAppenderResponse)

Writes the data to the specified Server, Database, Schema and table. It can also use expressions to define calculated columns.

In [ ]:
# Display the json configuration file
code $Path\Recipe_03_Output_Table_Expressions.json

In [ ]:
# Start XESmartTarget
Start-Process $env:ProgramFiles\XESmartTarget\xesmarttarget.exe -ArgumentList @("--File", "$Path\Recipe_03_Output_Table_Expressions.json", "--GlobalVariables","ServerName=$Server")

In [ ]:
# Display the data being captured
Invoke-Expression "$Path\Recipe_03_Display_Data.sql"

## Recipe4: Capture blocking and deadlocking, write to database and send email

One of the things that are missing from Extended Events is a mechanism for alerting. SQLTrace had Event Notifications (which is Service Broker on top of a trace), but Extended Events has nothing that comes close to it.

No problem: XESmartTarget has your back again! You can use an [EmailResponse](https://github.com/spaghettidba/XESmartTarget/wiki/EmailResponse) to send alerts whenever an interesting event is captured.

Moreover, you can can combine multiple responses together, so that the same events get forwarded to multiple responses.

In this example, I will write blocking and deadlocking events to a database table and send an email whenever these events are captured. I can also attach the deadlock graph or the blocking XML to the email message.

**Session:** Session04 - Blocking and deadlocking


In [ ]:
# Display the .sql script for the the session
code $Path\Recipe_04_Session_Setup.sql

In [ ]:
# Start the session
Invoke-DbaQuery -SqlInstance $Server -File "$Path\Recipe_04_Session_Setup.sql"

**Target**:  [TableAppenderResponse](https://github.com/spaghettidba/XESmartTarget/wiki/TableAppenderResponse) + [EmailResponse](https://github.com/spaghettidba/XESmartTarget/wiki/EmailResponse)



This configuration will write the events to a database table and will also notify deadlocks and blocking to an email address. 

I don't want to set up a real mailbox for sending alerts: in the real world I would use an open relay with the sql server's address in the allow list. In this case I will use [Papercut](https://github.com/ChangemakerStudios/Papercut-SMTP), a very handy tool that can be used to simulate a SMTP server and an email client, all collapsed in one single application. Setup is very easy and it comes very handy when demoing email alerting or troubleshooting email delivery.

In [ ]:
# Start Papercut
Start-Process "${env:ProgramFiles(x86)}\Changemaker Studios\Papercut SMTP\Papercut.exe"

In [ ]:
# Display the json configuration file
code $Path\Recipe_04_Alert_Email.json

In [ ]:
# Start XESmartTarget
Start-Process $env:ProgramFiles\XESmartTarget\xesmarttarget.exe -ArgumentList @("--File", "$Path\Recipe_04_Alert_Email.json", "--GlobalVariables","ServerName=$Server")

In [ ]:
# Generate some deadlocks
& $Path\Recipe_04_Generate_Deadlock.ps1 $Server

In [ ]:
# OK, enough deadlocks, thanks...
Get-Process | Where-Object {$_.ProcessName -eq "sqlcmd"} | Stop-Process

## Recipe5: Capture blocking and kill offending SPID

Notifications are appropriate when user intervention is required, but when the system can resolve the issue automatically it's better to avoid notifying users of the issue altogether.

In this case I would like to capture allo blocking sessions that have an open transaction but are not doing anything on the database. This can happen when the application is not designed correctly and performs heavy calculations on the client side mid-transaction or, even worse, leave open transactions beacause of flaws in the code (usually incorrect error handling).

Those sessions are usually safe to kill, especially if they are causing long standing blocking chains. XESmartTarget can do that too.

**Session:** Session05 - Blocking

In [ ]:
# Display the .sql script for the the session
code $Path\Recipe_05_Session_Setup.sql

In [ ]:
# Start the session
Invoke-DbaQuery -SqlInstance $Server -File "$Path\Recipe_05_Session_Setup.sql"

**Target**:  [ExecuteTSQLResponse](https://github.com/spaghettidba/XESmartTarget/wiki/ExecuteTSQLResponse)

This configuration will execute an arbitrary TSQL command every time an event is captured. In this case, I will use a stored procedure to kill the blocking session, passing the blocked session_id as a parameter.

In [ ]:
# Display the json configuration file
code $Path\Recipe_05_Kill_Blocking.json

In [ ]:
# Start XESmartTarget
Start-Process $env:ProgramFiles\XESmartTarget\xesmarttarget.exe -ArgumentList @("--File", "$Path\Recipe_05_Kill_Blocking.json", "--GlobalVariables","ServerName=$Server")

In [ ]:
# Display the stored procedure used to kill the blockers
code $Path\Recipe_05_KillBlocker.sql

In [ ]:
# Open two scripts to start blocking and getting blocked
Invoke-Item $Path\Recipe_05_Start_Blocking.sql
Invoke-Item $Path\Recipe_05_Get_Blocked.sql

## Recipe6: Audit Logins

When you inherit a big, busy and cahotic SQL Server instance, one of the thing that you probably want to do is track which logins are active and which ones are not and can be safely disabled.

One of the possible ways of doing this is to enable successful and failed logins in ERRORLOG, but this creates a lot of noise. No thanks, I don't want a messy ERRORLOG.

Another possibility is to capture login events with an Extended Events session and let it write all the events to a file target. However, with this approach you capture the individual events, which is not what you are interested in: what you really want is a very simple information: when has each login accessed the server the last time? Writing session data to a file target does not answer that question directly, but forces you to read and aggregate all the data in the file.

XESmartTarget can help with this, using the [GroupedTableAppenderResponse](https://github.com/spaghettidba/XESmartTarget/wiki/GroupedTableAppenderResponse). This Response type aggregates the data in memory before writing it to a target table, where it is merged with the existing data.

**Session:** Recipe06 - Logins

In [ ]:
# Display the .sql script for the the session
code $Path\Recipe_06_Session_Setup.sql

In [ ]:
# Start the session
Invoke-DbaQuery -SqlInstance $Server -File "$Path\Recipe_06_Session_Setup.sql"

**Target:**  [GroupedTableAppenderResponse](https://github.com/spaghettidba/XESmartTarget/wiki/GroupedTableAppenderResponse)

This target will take all non aggregated columns and use them as the key to merge the data with the existing rows in the table.

In [ ]:
# Display the json configuration file
code $Path\Recipe_06_Login_Audit.json

In [ ]:
# Start XESmartTarget
Start-Process $env:ProgramFiles\XESmartTarget\xesmarttarget.exe -ArgumentList @("--File", "$Path\Recipe_06_Login_Audit.json", "--GlobalVariables","ServerName=$Server")

In [ ]:
# Display the data captured by XESmartTarget
Invoke-Item $Path\Recipe_06_Display_Data.sql

## Recipe7: Unused Objects

The obvious way to detect which objects are used and which ones are safe to delete is to read the database documentation. Unfortunately this only works in the fantasy world where database documentation exists and is updated regularly.

The second best option is setting up a database audit and capture all activity on the server. Unfortunately this is tedious to set up and not so easy to automate.

Extended Events is easier to automate, so let's do that instead. Extended Events is the foundation of audits, but you can't access the events used by audits directly in your sessions. That's a shame.

You can use other events instead of the private events for auditing. [I blogged about this topic some years ago](https://spaghettidba.com/2015/04/20/tracking-table-usage-and-identifying-unused-objects).


**Session:** Recipe07 - Locks



In [ ]:
# Display the .sql script for the the session
code $Path\Recipe_07_Session_Setup.sql

In [ ]:
# Start the session
Invoke-DbaQuery -SqlInstance $Server -File "$Path\Recipe_07_Session_Setup.sql"

**Target**:  [GroupedTableAppenderResponse](https://github.com/spaghettidba/XESmartTarget/wiki/GroupedTableAppenderResponse).

In [ ]:
# Display the json configuration file
code $Path\Recipe_07_Table_Audit.json

In [ ]:
# Start XESmartTarget
Start-Process $env:ProgramFiles\XESmartTarget\xesmarttarget.exe -ArgumentList @("--File", "$Path\Recipe_07_Table_Audit.json", "--GlobalVariables","ServerName=$Server")

In [ ]:
# Display the data captured by XESmartTarget
Invoke-Item $Path\Recipe_07_Display_Data.sql

## Recipe8: Workload Analysis

The idea comes from a blog post by Brent Ozar about "[How to Find Out Whose Queries are Using The Most CPU](https://www.brentozar.com/archive/2020/08/how-to-find-out-whose-queries-are-using-the-most-cpu/)". Brent uses the Resource Governor to detect who's using the CPU. That's an interesting approach, but you can do the same with XESmartTarget.

For this demo I will capture execution events (`rpc_completed` and `sql_batch_completed`) and I will aggregate the data using the [GroupedTableAppenderResponse](https://github.com/spaghettidba/XESmartTarget/wiki/GroupedTableAppenderResponse).

**Session**: Recipe08 - Execution Events



In [ ]:
# Display the .sql script for the the session
code $Path\Recipe_08_Session_Setup.sql

In [ ]:
# Start the session
Invoke-DbaQuery -SqlInstance $Server -File "$Path\Recipe_08_Session_Setup.sql"

**Target**:  [GroupedTableAppenderResponse](https://github.com/spaghettidba/XESmartTarget/wiki/GroupedTableAppenderResponse).

In [ ]:
# Display the json configuration file
code $Path\Recipe_08_Workload_Analysis.json

In [ ]:
# Start XESmartTarget
Start-Process $env:ProgramFiles\XESmartTarget\xesmarttarget.exe -ArgumentList @("--File", "$Path\Recipe_08_Workload_Analysis.json", "--GlobalVariables","ServerName=$Server")

In [ ]:
# Display the data captured by XESmartTarget
Invoke-Item $Path\Recipe_08_Display_Data.sql

In [ ]:
# Start Grafana
Start-Process powershell.exe -ArgumentList @("-Command","Start-Service Grafana") -Verb Runas

In [ ]:
# Display the data in a Grafana dashboard
Start-Process msedge.exe -ArgumentList @("http://localhost:3000/d/okg3se6Mk/workload-analysis?orgId=1", "-inPrivate")

## Recipe9: Write events to Influxdb database

As you have see, Grafana is a very powerful tool for displaying time series. It can be combined with a time series database like Influxdb and a collection agent like Telegraf to create a monitoring solution.

However, Telegraf is capable of polling data from several DMVs at regular intervals, but out of the box it cannot intercept events from the server. Again, XESmartTarget has got you covered.

The [TelegrafAppenderResponse](https://github.com/spaghettidba/XESmartTarget/wiki/TelegrafAppenderResponse) is designed to let XESmartTarget print the data it collects to stdout, using a special syntax understood by telegraf and InfluxDB. This is called the InfluxDB line protocol: for each data point it presents the target measurement, the timestamp (remember, this is a time series database) and then tags (indexed columns) and fields (unindexed columns that contain the metrics).

**Session**: Recipe09 - Errors, blocking and deadlocks

In [ ]:
# Display the .sql script for the the session
code $Path\Recipe_09_Session_Setup.sql

In [ ]:
# Start the session
Invoke-DbaQuery -SqlInstance $Server -File "$Path\Recipe_09_Session_Setup.sql"


**Target**:  [TelegrafAppenderResponse](https://github.com/spaghettidba/XESmartTarget/wiki/TelegrafAppenderResponse)

In [ ]:
# Display the json configuration file
code $Path\Recipe_09_Telegraf.json

In [ ]:
# Start influxdb
Start-Process "C:\tick\influxdb-1.7.9-1\influxd.exe"

In [ ]:
# Start telegraf
Start-Process $Path\telegraf.exe -ArgumentList @("--config", "$Path\Recipe_09_telegraf.conf")

In [ ]:
# Generate some deadlocks
& $Path\Recipe_04_Generate_Deadlock.ps1 $Server

In [ ]:
# Display the data in a Grafana dashboard
Start-Process msedge.exe -ArgumentList @("http://localhost:3000/d/6VbiagVGz/sql-server-events?orgId=1", "-inPrivate")

In [ ]:
# OK, enough deadlocks, thanks...
Get-Process | Where-Object {$_.ProcessName -eq "sqlcmd"} | Stop-Process